In [ ]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:100]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 1  # Adjust this number based on your data
n_nonzero_coefs = 5  # Number of non-zero coefficients in sparse coding

In [3]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs)
dict_learner.fit(train_data_combined)
D_learned = dict_learner.components_  # Shape: (n_components, feature_dim)


# Transform training data into sparse codes
train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))

print('Training data shape:', train_alice_sparse_codes.shape)

Training data shape: (3200, 1)


In [3]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))


# Transform test data into sparse codes
test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# Combine the sparse codes
test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
print('Test data shape:', test_sparse_codes.shape)

Test data shape: (1600, 1)


In [4]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


In [5]:
# Initialize the classifier
classifier = LogisticRegression(max_iter=1000)

# Train the classifier on the sparse codes
classifier.fit(train_sparse_codes, train_labels)


LogisticRegression(max_iter=1000)

In [6]:
# Predict labels for test data
predictions = classifier.predict(test_sparse_codes)

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 58.00%
tp: 488
tn: 440
fp: 360
fn: 312
MDR: 0.45
FAR: 0.39
AR: 0.58
